
# Setup for Pangeo Environment on Gadi


In this notebook we will go through:

- Configuring the Pangeo environment using your Gadi account
- Submitting and monitoring multi-node Pangeo jobs to Gadi
- Running Pangeo Jupyter notebooks in batch mode non-interactively

The [Pangeo](http://pangeo.io) software ecosystem involves open source tools such as [xarray](http://xarray.pydata.org/en/stable/), [iris](https://scitools.org.uk/iris/docs/latest/), [dask](https://dask.org/), [Jupyter](https://jupyter.org/) and many other packages. 

This notebook provides instructions on how to use the Pangeo environment to run a multi-node parallel Jupyter notebook within the queues on Gadi and shows how to interact with it from your desktop.


## Configuring your account on Gadi

###  Step 1: Enabling Pangeo in your shell envorinment

To enable the Pangeo environment, you can use the following command within jobs, or within an interactive environment:

```
$ module load pangeo/2019.12
        Loading requirement: intel-mkl/2019.3.199 python3/3.7.4 hdf5/1.10.5 netcdf/4.7.1

```
Note that Pangeo has its own Python installation.


###  Step 2: Configuring your JupyterLab password on Gadi

We will use JupyterLab to load notebooks and monitor jobs. JupyterLab is bundled within the Pangeo environment. To setup this environment, run the following two commands:

```
$ jupyter notebook --generate-config
$ jupyter notebook password
```

This is a stand-alone password that you will use later for accessing the JupyterLab server. You can use this command to reset your password at any time.

### Step 3: Exiting the Pangeo environment

You can quit the environment at any time by running:
```
$pangeo.end.sh
```

### Submitting a multi-node Pangeo job to Raijin

First create a directory where you will run the jupyter notebook:

```
$ mkdir -p ~/pangeo/tutorial

```
You can create a shell script by copying the following commands into a script file. Let’s name this file run_ipynb_job.sh:

```
#!/bin/bash
#PBS -N pangeo_test
#PBS -P c25
#PBS -q normal
#PBS -l walltime=5:00:00
#PBS -l ncpus=96
#PBS -l mem=384GB
#PBS -l jobfs=100GB
#PBS -l storage=scratch/z00+scratch/c25+gdata/c25
module load pangeo/2019.12
pangeo.ini.all.sh
sleep infinity
```

Note that for Gadi, the "#PBS -l storage=" flag is required which must include **all** the scratch and gdata project IDs that will be used in your codes. 

Additionally, jobs beyond a single node must use multiples of 48 for their ncpus request. 

Dask requires whole nodes so you need to have a job that requires these resources. In this example we request 2 Gadi Cascade Lake nodes with 96 CPUs and 384GB memory. For further instructions about node types on Gadi, consult the [NCI user guide](https://opus.nci.org.au/display/Help/Gadi+Job+Queues) on the specific configurations. 


The above jobscript will load the Pangeo module, run the initialisation script called **pangeo.ini.all.sh**, and keep the environment alive for the lifetime of the PBS job. The initialisation script will set up the dask scheduler on one node and multiple workers on all nodes.

To submit the job to the queue:

```
$ qsub run_ipynb_job.sh

```

and take note of your job_id (which may look something like 1030967.gadi-pbs). Check the queue to see when the job is running:

```
$ qstat <job_id>

```


![3](images/pangeo_setup3_gadi.png)



### Set up port forwarding to access from your desktop machine

Once the job is running on Gadi, there will be two files that are created in your current Gadi directory: 

* client_cmd
* scheduler.json

The file client_cmd contains commands to forward network traffic from the defined port number of the worker node to an external client machine (i.e., your desktop) via the login node gadi.nci.org.au. 

Running
```
$ more client_cmd

```
should give an output that looks something like:

```
ssh -N -L 8388:gadi-cpu-clx-2224.gadi.nci.org.au:8388 abc123@gadi.nci.org.au
ssh -N -L 8768:gadi-cpu-clx-2224.gadi.nci.org.au:8768 abc123@gadi.nci.org.au 

```
For this example, JupyterLab uses port 8388 and dask dashboard occupies port 8768 respectively from Gadi worker node gadi-cpu-clx-2224. 

Note that both port numbers are randomly chosen for each job and will be different each time you run a new job.

Next, on your remote machine (e.g., Desktop), run each of the following commands separately and enter your Gadi password if prompted.

CLI_1: 

```
$ ssh -N -L 8388:gadi-cpu-clx-2224.gadi.nci.org.au:8388 abc123@gadi.nci.org.au &

```

CLI_2: 

```
$ ssh -N -L 8768:gadi-cpu-clx-2224.gadi.nci.org.au:8768 abc123@gadi.nci.org.au &

```

### Connect to the remote JupyterLab server from your remote desktop computer

In a browser on your local desktop, type the following URL “localhost:8388” – where the port matches the details above:

![6](images/pangeo_setup6_gadi.png)

You will be prompted for your JupyterLab password that you created earlier:
![7](images/pangeo_setup7.png)

Once your authentication has passed, a JupyterLab interface will be launched in a few seconds.

![8](images/pangeo_setup8.png)

### Importing a notebook example

You can drag and drop a notebook from your local machine into the JupyterLab. This file will then also appear in your working directory on Gadi:
![9](images/pangeo_setup9.png)

The screen shot above shows

1. Jupyter notebook interface opened from your local computer and connecting to Gadi
2. A directory on your local machine from where a notebook is dragged and dropped into the JupyterLab interface
3. Gadi command window showing that the notebook appears instantly

### Setting up your Jupyter notebook to use the dask server

**At the beginning of the notebook**

To utilise the dask server established from the PBS job, it is necessary to add and run the following cell at the beginning of your notebook: 

```
from dask.distributed import Client,LocalCluster 
client = Client(scheduler_file='scheduler.json') 
print(client) 
```

The output will show the configuration of the client and dask cluster. You can check that the number of cores matches what you requested in the job script. Now you could run your notebook as usual.

**At the end of the jobscript**

Add and run a cell as below to gracefully stop the job:

```
!pangeo.end.sh
```

### Recap important notes

Please make sure the following start and stop instructions are added at the beginning and the end of the notebook respectively.

```
# start the dask client
client =  Client(scheduler_file='scheduler.json')
 
# stop the pbs job.
!pangeo.end.sh
```

### View compute threads using Dask dashboard

From your local desktop, open a new tab in the web browser and type the second port in the client_cmd file to open the Dask dashboard. This will allow you to see the dynamic resources of the processing. In your Jupyter notebook, a computationally intensive task will be recognised with a (*) at the start of a cell that is currently in execution.  In the Dask dashboard you would then see something similar to the following:

```
localhost:8768
```

![10](images/pangeo_setup10.png)

### Run Jupyter notebook in a batch job

Of course you don’t need to run Jupyter notebooks interactively. You can just submit them as a job.

### Step 1: Convert your Jupyter notebook to a Python script

```
$ module load pangeo

$ jupyter nbconvert --to script [YOUR_NOTEBOOK}.ipynb
```

Make sure you have added the following lines to the beginning of the Python script.

```
from dask.distributed import Client,LocalCluster 
client = Client(scheduler_file='scheduler.json') 
print(client) 
```


### Step 2: Create the job script

```
#!/bin/bash 
#PBS -N pangeo_test 
#PBS -q normal
#PBS -P c25
#PBS -l walltime=5:00:00 
#PBS -l ncpus=96 
#PBS -l mem=384GB 
#PBS -l jobfs=100GB 
#PBS -l storage=scratch/c25+gdata/c25
 
module load pangeo/2019.10 
pangeo.ini.all.sh 
 
cd $PBS_O_WORKDIR 
python3 YOUR_PYSCRIPT_NAME.py 
pangeo.end.sh
```

Modify the parameters to suit your code and give it a name (e.g. **run_py.sh**)

### Step 3: Submit your job script via 'qsub' command

```
$ qsub run_py.sh
```


Note that users need to manage all necessary modules by themselves.

### Reference

- http://pangeo.io